<a href="https://colab.research.google.com/github/RogerHeederer/DLofAll/blob/master/mnist_xavier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!git clone https://github.com/RogerHeederer/DLofAll.git

Cloning into 'DLofAll'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [3]:
#파라미터
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

In [5]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     drop_last=True)

In [12]:
# 레이어 쌓기
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [13]:
#레이어 연결 + 이니셜라이져 설정
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.1071,  0.0212,  0.0940,  ..., -0.0250,  0.0036,  0.0741],
        [ 0.0701, -0.0980,  0.0392,  ..., -0.0257, -0.0156,  0.0655],
        [ 0.0462,  0.0101,  0.0576,  ..., -0.0536, -0.0075,  0.0753],
        ...,
        [ 0.0824, -0.0140,  0.0941,  ..., -0.0864,  0.0665,  0.0617],
        [-0.0229,  0.0500,  0.0208,  ..., -0.0134, -0.0696, -0.0943],
        [-0.0334,  0.0881,  0.0705,  ...,  0.0328, -0.0301,  0.0670]],
       requires_grad=True)

In [14]:
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

In [15]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
total_batch = len(data_loader)
model.train()
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost/total_batch

  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.293597728
Epoch: 0002 cost = 0.145064294
Epoch: 0003 cost = 0.112555712
Epoch: 0004 cost = 0.093380317
Epoch: 0005 cost = 0.082439244
Epoch: 0006 cost = 0.074956633
Epoch: 0007 cost = 0.071431108
Epoch: 0008 cost = 0.061949197
Epoch: 0009 cost = 0.058488585
Epoch: 0010 cost = 0.052778292
Epoch: 0011 cost = 0.051322408
Epoch: 0012 cost = 0.048464682
Epoch: 0013 cost = 0.046216615
Epoch: 0014 cost = 0.043253243
Epoch: 0015 cost = 0.044716623
Learning finished


In [22]:
with torch.no_grad():
  model.eval()

  X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)# view는 reshape
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  r = random.randint(0, len(mnist_test)-1)
  X_single_data = mnist_test.test_data[r].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.test_labels[r].to(device)

  print('label: ', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction: ', torch.argmax(single_prediction, 1).item())

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9801999926567078
label:  8
Prediction:  8


In [25]:
single_prediction

tensor([[   32.0517,    95.8327,    95.8128,   507.5999, -1076.4242,  -842.3981,
         -1031.0350,  -665.5845,   981.7748,  -172.2188]])

In [28]:
torch.argmax(single_prediction, 1).item()

8

In [29]:
Y_single_data

tensor(8)

In [30]:
Y_single_data.item()

8